In [ ]:
import os
import imgaug.augmenters as iaa
import cv2

import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from multiprocessing import Pool
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras import layers, activations
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
import seaborn as sns
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def copie_images(s,d,n):
    os.makedirs(d, exist_ok=True)
    files = os.listdir(s)
    image_files = [file for file in files if file.lower().endswith(('.png','.PNG','JPG','jpg'))]
    for image in image_files[:n]:
        src_path = os.path.join(s, image)
        dest_path = os.path.join(d, image)
        shutil.copy(src_path, dest_path)
    print("Copie terminée.")

In [ ]:
def count_images_in_directory(directory):
  num_images = 0
  for filename in os.listdir(directory):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp','.JPG','PNG')):
       num_images += 1

  return num_images

In [ ]:


def load_images_from_folder(folder, num_images=100):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            if len(images) >= num_images:
                break
    return images

def display_images(images):
    num_images = len(images)
    cols = 10# Number of columns for the grid display
    rows = num_images // cols + (num_images % cols > 0)  # Compute number of rows needed

    plt.figure(figsize=(20, 20))  # Adjust figure size as needed
    for i, img in enumerate(images):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img_rgb)
        plt.axis('off')
    plt.show()




In [ ]:
input_folder = '/kaggle/input/base-copiedelocale-cropped-image/Croped_image_male_femelle/femelle'
num_images_to_display = 100

images = load_images_from_folder(input_folder, num_images_to_display)
display_images(images)

In [ ]:
male='/kaggle/input/base-copiedelocale-cropped-image/Croped_image_male_femelle/male'
m=count_images_in_directory(male)


femelle='/kaggle/input/base-copiedelocale-cropped-image/Croped_image_male_femelle/femelle'
f=count_images_in_directory(femelle)
print("male", m)
print("femelle",f)

In [ ]:
import plotly.graph_objects as go

labels = ["male", "femelle"]
values = [m, f]
colors = ["gold", "mediumturquoise"]

fig = go.Figure(
    data=[
        go.Pie(
            labels=labels,
            values=values,
            textfont_size=20,
            marker=dict(colors=colors, pattern=dict(shape=[".", "x"]))
        )
    ]
)

fig.show()

In [ ]:

import random

def zoom_image(image, zoom_factor):
    height, width = image.shape[:2]
    new_height, new_width = int(height / zoom_factor), int(width / zoom_factor)
    start_row = (height - new_height) // 2
    start_col = (width - new_width) // 2
    end_row = start_row + new_height
    end_col = start_col + new_width
    cropped_image = image[start_row:end_row, start_col:end_col]
    zoomed_image = cv2.resize(cropped_image, (width, height), interpolation=cv2.INTER_LINEAR)
    return zoomed_image

def add_gaussian_noise(image, mean=0, stddev=25):
    gauss = np.random.normal(mean, stddev, image.shape).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image

def translate_image_random_fill(image, x, y):
    h, w = image.shape[:2]
    M = np.float32([[1, 0, x], [0, 1, y]])
    translated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    if x > 0:
        translated[:, :x] = np.random.randint(0, 256, (h, x, 3), dtype=np.uint8)
    elif x < 0:
        translated[:, x:] = np.random.randint(0, 256, (h, -x, 3), dtype=np.uint8)
    if y > 0:
        translated[:y, :] = np.random.randint(0, 256, (y, w, 3), dtype=np.uint8)
    elif y < 0:
        translated[y:, :] = np.random.randint(0, 256, (-y, w, 3), dtype=np.uint8)
    return translated




def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated


def random_rotate_image(image, min_angle=-20, max_angle=20):
    angle = random.uniform(min_angle, max_angle)
    return rotate_image(image, angle)


def adjust_brightness(image, value):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float32)
    hsv[:, :, 2] += value
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    hsv = hsv.astype(np.uint8)
    brightened_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return brightened_image

def flip_image(image, flip_code):
    return cv2.flip(image, flip_code)


def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def process_and_save_images(input_folder, output_folder, c):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    augmentations = [
        lambda img: zoom_image(img, random.uniform(1.1, 1.5)),
        lambda img: translate_image_random_fill(img, random.randint(-20, 20), random.randint(-20, 20)),
        lambda img: adjust_brightness(img, random.randint(-50, 50)),
        lambda img: flip_image(img, 0),  # Horizontal flip
        lambda img: flip_image(img, 1),  # Vertical flip
        lambda img: random_rotate_image(img),
        lambda img: convert_to_grayscale(img)  # Convertir en niveaux de gris
    ]

    for i, file_name in enumerate(image_files):
        img = cv2.imread(os.path.join(input_folder, file_name))
        img_name, img_ext = os.path.splitext(file_name)

        # Appliquer chaque augmentation 14 fois
        for aug in augmentations:
            for j in range(c):
                aug_img = aug(img.copy())
                # Vérifier si l'image est en niveaux de gris et ajuster le nom de fichier
                if len(aug_img.shape) == 2:  # L'image est en niveaux de gris
                    output_file_name = f"{img_name}_gray_aug_{j}{img_ext}"
                else:
                    output_file_name = f"{img_name}_aug_{j}{img_ext}"
                
                cv2.imwrite(os.path.join(output_folder, output_file_name), aug_img)


In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def remove_dir(path):
     os.system(f'rm -rf {path}')

In [ ]:
create_dir("/kaggle/working/gray_Image/M")
create_dir("/kaggle/working/gray_Image/f")

In [ ]:
input_folder = '/kaggle/input/base-copiedelocale-cropped-image/Croped_image_male_femelle/femelle'
output_folder = '/kaggle/working/gray_Image/f'
process_and_save_images(input_folder, output_folder,3)

In [ ]:
input_folder = '/kaggle/input/base-copiedelocale-cropped-image/Croped_image_male_femelle/male'
output_folder = '/kaggle/working/gray_Image/M'
process_and_save_images(input_folder, output_folder,3)

In [ ]:
male='/kaggle/working/gray_Image/M'
m=count_images_in_directory(male)


femelle='/kaggle/working/gray_Image/f'
f=count_images_in_directory(femelle)
print("male",m)
print("femelle",f)

In [ ]:
create_dir("/kaggle/working/gray_Image/femelle")

In [ ]:
create_dir("/kaggle/working/gray_Image/male")

In [ ]:
n=2748
s = '/kaggle/working/gray_Image/M'
d ='/kaggle/working/gray_Image/male'
copie_images(s,d,n)

In [ ]:
remove_dir("/kaggle/working/gray_Image/M")

In [ ]:
n=2748
s = '/kaggle/working/gray_Image/f'
d ='/kaggle/working/gray_Image/femelle'
copie_images(s,d,n)

In [ ]:
 remove_dir("/kaggle/working/gray_Image/f")

In [ ]:
male='/kaggle/working/gray_Image/male'
m=count_images_in_directory(male)


femelle='/kaggle/working/gray_Image/femelle'
f=count_images_in_directory(femelle)
print("male",m)
print("femelle",f)

In [ ]:
import plotly.graph_objects as go

labels = ["male", "femelle"]
values = [m, f]
colors = ["gold", "mediumturquoise"]

fig = go.Figure(
    data=[
        go.Pie(
            labels=labels,
            values=values,
            textfont_size=20,
            marker=dict(colors=colors, pattern=dict(shape=[".", "x"]))
        )
    ]
)

fig.show()

In [ ]:
pip install split-folders

In [ ]:
create_dir("/kaggle/working/BaseSplited")

In [ ]:
import splitfolders  

input_folder = "/kaggle/working/gray_Image"  
output_folder = '/kaggle/working/BaseSplited'  

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.7, .3))

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight

train_dir = '/kaggle/working/BaseSplited/train'
val_dir = '/kaggle/working/BaseSplited/val'


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(600, 600),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(600, 600),
    batch_size=batch_size,
    class_mode='binary'
)


In [ ]:
import warnings
import os
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [ ]:
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  

# Définir le modèle  
model1 = Sequential([  
    # Première série de couches Convolution + MaxPooling
    Conv2D(128, (7, 7), activation='relu', padding='same', input_shape=(600, 600, 3)),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),  
    
    Conv2D(128, (7, 7), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),
    
    # Deuxième série de couches Convolution + MaxPooling
    Conv2D(64, (5, 5), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),  
    
    Conv2D(64, (5, 5), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)), 
    
    # Troisième série de couches Convolution + MaxPooling
    Conv2D(64, (3, 3), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),  
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),  
    BatchNormalization(),  
    MaxPooling2D((2, 2)),  
    
    # Couche Flatten
    Flatten(),  
    
    # Première couche fully connected
    Dense(512, activation='relu'),  
    Dropout(0.5),  
    
    # Deuxième couche fully connected
    Dense(256, activation='relu'),  
    Dropout(0.5),  
    
    # Troisième couche fully connected
    Dense(128, activation='relu'),  
    Dropout(0.5),  
    
    # Couche de sortie
    Dense(1, activation='sigmoid')  # 'softmax' pour plus de deux classes  
])  

model1.compile(optimizer='adam',  
              loss='binary_crossentropy',  # 'categorical_crossentropy' pour plus de deux classes  
              metrics=['accuracy'])  

# Callbacks avec la bonne extension  
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  
model_checkpoint = ModelCheckpoint('_model1.keras', save_best_only=True)  # Changez l'extension ici  

# Résumé du modèle  
model1.summary()


In [ ]:
# Calculer les poids des classes
y_true = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_true), y=y_true)
class_weights_dict = dict(enumerate(class_weights))


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  
model_checkpoint = ModelCheckpoint('_model1.keras', save_best_only=True)  

history3 = model1.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint]
)


y_pred = model1.predict(val_generator)
y_pred_classes = (y_pred > 0.5).astype("int32").flatten()
y_true = val_generator.classes

balanced_accuracy = balanced_accuracy_score(y_true, y_pred_classes)
print(f'Balanced Accuracy: {balanced_accuracy}')

print(classification_report(y_true, y_pred_classes))

In [ ]:
# y_true = train_generator.classes
# class_weights = compute_class_weight('balanced', classes=np.unique(y_true), y=y_true)
# class_weights_dict = dict(enumerate(class_weights))


# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  
# model_checkpoint = ModelCheckpoint('_model1.keras', save_best_only=True)  


In [ ]:
print(y_true)

In [ ]:
print(class_weights )

In [ ]:
class_weights_dict

In [ ]:
early_stopping

In [ ]:
model_checkpoint

In [ ]:
y_pred

In [ ]:
y_pred_classes

In [ ]:
y_true

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Charger un lot d'images de test
test_images, test_labels = next(val_generator)

# Afficher quelques images avec leurs étiquettes
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Label: {int(test_labels[i])}")
    plt.axis('off')
plt.show()


In [ ]:
import os
import random
from PIL import Image

def display_random_images(directory, num_images=5):
    all_images = os.listdir(directory)
    random_images = random.sample(all_images, num_images)
    
    plt.figure(figsize=(10, 10))
    for i, img_name in enumerate(random_images):
        img_path = os.path.join(directory, img_name)
        img = Image.open(img_path)
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.title(img_name)
        plt.axis('off')
    plt.show()

# Afficher des images aléatoires pour vérifier les étiquettes
display_random_images('/kaggle/working/BaseSplited/val/male')
display_random_images('/kaggle/working/BaseSplited/val/femelle')


In [ ]:
# Inspecter les premières images et leurs étiquettes dans le générateur de données
for data_batch, labels_batch in train_generator:
    # Afficher les premières images et leurs étiquettes
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(data_batch[i])
        plt.title(f"Label: {int(labels_batch[i])}")
        plt.axis('off')
    plt.show()
    break  # Sortir après la première itération pour éviter de traiter tout le batch


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

def verify_labels(directory):
    datagen = ImageDataGenerator()
    generator = datagen.flow_from_directory(
        directory,
        target_size=(600, 600),
        batch_size=1,  # Utiliser un petit batch size pour la vérification
        class_mode='binary'  # Supposons une classification binaire
    )
    
    for batch in generator:
        images, labels = batch
        # Vérifier la forme de labels pour comprendre la structure
        print(f"Labels shape: {labels.shape}")

        # Si labels est un vecteur one-hot ou une matrice
        if labels.ndim > 1:
            for i in range(len(labels)):
                print(f"Label: {np.argmax(labels[i])}")  # Trouver l'index de la classe
        else:
            # Si labels est une valeur scalaire
            for i in range(len(labels)):
                print(f"Label: {int(labels[i])}")

        plt.imshow(images[0])
        plt.axis('off')
        plt.show()
        break  # Sortir après la première image pour vérification

# Vérifiez les répertoires de train et de validation
verify_labels('/kaggle/working/BaseSplited/train')
verify_labels('/kaggle/working/BaseSplited/val')


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Prédire les étiquettes pour les données de test
test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(600, 600),
    batch_size=1,  # Utiliser un petit batch size pour la vérification
    class_mode='binary',
    shuffle=False  # Important pour garder l'ordre des échantillons
)

predictions = model1.predict(test_generator, steps=test_generator.samples, verbose=1)
predicted_labels = (predictions > 0.5).astype('int32')  # Binariser les prédictions

# Obtenir les véritables étiquettes
true_labels = test_generator.classes

# Calculer la matrice de confusion
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)

# Afficher le rapport de classification
report = classification_report(true_labels, predicted_labels, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)


In [ ]:
# Obtenir les véritables étiquettes
true_labels = test_generator.classes

# Calculer la matrice de confusion
cm = confusion_matrix(true_labels, predicted_labels)

# Afficher la matrice de confusion avec Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Prédire les probabilités pour les données de test
probabilities = model1.predict(test_generator, steps=test_generator.samples, verbose=1)

# Obtenir les véritables étiquettes
true_labels = test_generator.classes

# Calculer les courbes ROC
fpr, tpr, thresholds = roc_curve(true_labels, probabilities)

# Calculer l'AUC
roc_auc = auc(fpr, tpr)

# Afficher la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:

test_loss, test_accuracy = model1.evaluate(test_generator)

# Affichage des courbes de précision
plt.figure(figsize=(12, 4))

# Précision d'entraînement, de validation et de test
plt.subplot(1, 2, 1)
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.axhline(y=test_accuracy, color='r', linestyle='--')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation', 'Test'], loc='upper left')

# Affichage des courbes de perte (loss)
plt.subplot(1, 2, 2)
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.axhline(y=test_loss, color='g', linestyle='--')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation', 'Test'], loc='upper left')

plt.show()
